# SRTM Bore Plotting Notebook - SC0-SC1-SC2
This notebook dynamically generates plots for bore data and saves them into an output folder.


### Groundwater Analysis Project Using Bore Data and SRTM Model

Groundwater analysis project that integrates SRTM (Shuttle Radar Topography Mission) data with borehole monitoring for more accurate modeling and management of groundwater resources. This project aims to understand how terrain elevation influences groundwater behavior and to analyze water levels and quality under various conditions.

Key Components of the Project:

* SRTM Data Integration:

SRTM data provides detailed terrain elevation information, which is used to assess how the landscape affects groundwater flow and aquifer recharge.
By understanding terrain elevation, Mathews can model how water moves across and within the landscape, impacting where and how quickly water infiltrates into the aquifer.
This integration allows for a more comprehensive analysis of the relationship between topography and groundwater dynamics, which is crucial for sustainable water resource management.
Borehole Monitoring and Conditions: Mathews is analyzing different borehole conditions to monitor water levels, water quality, and predict future trends. The conditions being tracked are:

* SC0 (No Injection/Natural Condition):

This represents the baseline natural state of the boreholes or aquifer with no external interventions (injection).
It provides a reference point to compare the effects of human activities or natural events.

* SC1 (Existing Injection Condition):

SC1 monitors the current state of the aquifer where injection is happening (e.g., injecting water, chemicals, or other substances into the borehole).
This condition helps assess how the aquifer is responding to ongoing or recent injections, such as changes in water levels or quality.

* SC2 (Prediction Condition):

SC2 is used to predict future outcomes for the aquifer based on the current state and trends observed in SC0 and SC1.
This condition is crucial for forecasting how the aquifer will behave in the future, such as water level recovery after injections stop or the spread of contaminants.
Predictive models like SC2 help in planning sustainable groundwater management strategies.

* Data Collection and Analysis:

Water level and quality data are collected over time from various boreholes. This data is visualized using time series plots to understand trends and variations.
The project includes field observations and automated time series measurements to ensure accurate and comprehensive monitoring of groundwater conditions.
Purpose of the Analysis:

The primary goal of this project is to provide insights into the behavior of the aquifer system under both natural and human-influenced conditions.

By integrating SRTM terrain data and borehole water-level monitoring, the project aims to:

* Predict future water levels and quality.

* Assess the sustainability of current water management practices, such as injection schemes.

* Help in decision-making for water resource managers by forecasting long-term impacts and providing a better understanding of the aquifer’s response to interventions.

* Identify risks associated with aquifer depletion or contamination and develop strategies for mitigating those risks.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

# Load the merged main data from the Excel file
merged_main_data_path = 'Merged_Main_Data.xlsx'
merged_data_df = pd.read_excel(merged_main_data_path)

# Create a directory for saving the plots
output_dir = 'bore_plots_final_2_dynamic'
os.makedirs(output_dir, exist_ok=True)

# Extract unique bore IDs for SC0, SC1, SC2, and TimeSeries (TS) columns
sc_columns = [col for col in merged_data_df.columns if 'SC0' in col or 'SC1' in col or 'SC2' in col]

# Extract unique bores
unique_bores = list(set([col.split(' ')[0] for col in sc_columns]))

# Option 2: Sort the bores according to their order of appearance in the Excel file
#unique_bores = list(dict.fromkeys([col.split(' ')[0] for col in merged_data_df.columns if 'SC0' in col or 'SC1' in col or 'SC2' in col]))

# Function to plot bore data on subplots for each page
def plot_bore_data_subplot(bores_to_plot, page_number):
    valid_bores = []
    
    # First, filter bores with actual data
    for bore_id in bores_to_plot:
        sc0_data = merged_data_df.get(f'{bore_id} SC0')
        sc1_data = merged_data_df.get(f'{bore_id} SC1')
        sc2_data = merged_data_df.get(f'{bore_id} SC2')
        ts_column = f'{bore_id}_TS'
        field_obs_column = f'{bore_id}_FV'
        
        # Check if there's any data for the bore
        if sc0_data is not None or sc1_data is not None or sc2_data is not None or ts_column in merged_data_df.columns or field_obs_column in merged_data_df.columns:
            valid_bores.append(bore_id)

    # Now that we know how many bores to plot, dynamically adjust the number of subplots
    n_bores = len(valid_bores)
    if n_bores == 0:
        print(f"No valid bores to plot on page {page_number}.")
        return
    
    # Calculate the number of rows needed based on the number of valid bores (max 6 per page, in a 3x2 grid)
    rows = math.ceil(n_bores / 2)
    fig, axes = plt.subplots(rows, 2, figsize=(8.27, 11.69))  # Create a dynamic grid of subplots

    if rows == 1:
        axes = [axes]  # If only one row, make it iterable

    fig.subplots_adjust(left=0.1, right=0.95, top=0.95, bottom=0.1, hspace=0.5, wspace=0.4)

    # Loop through the valid bores and plot them
    for i, bore_id in enumerate(valid_bores):
        row, col = divmod(i, 2)
        ax = axes[row][col] if rows > 1 else axes[col]  # Handle single row case

        # Plot SC0, SC1, SC2 data if available
        dates = pd.to_datetime(merged_data_df['Date'], dayfirst=True)
        sc0_data = merged_data_df.get(f'{bore_id} SC0')
        sc1_data = merged_data_df.get(f'{bore_id} SC1')
        sc2_data = merged_data_df.get(f'{bore_id} SC2')

        if sc0_data is not None:
            ax.plot(dates, sc0_data, label='SC0', color='blue')
        if sc1_data is not None:
            ax.plot(dates, sc1_data, label='SC1', color='green')
        if sc2_data is not None:
            ax.plot(dates, sc2_data, label='SC2', color='orange')

        # Plot Time Series data if it exists
        ts_column = f'{bore_id}_TS'
        if ts_column in merged_data_df.columns:
            ts_data = merged_data_df[ts_column]
            ax.plot(dates, ts_data, label='Obs (Time Series)', color='purple', linestyle='-', linewidth=0.6)  # Thinner line

        # Plot Field Observation data if it exists
        field_obs_column = f'{bore_id}_FV'
        if field_obs_column in merged_data_df.columns:
            field_obs_data = merged_data_df[field_obs_column]
            ax.scatter(dates, field_obs_data, label='Obs (Field Visit)', color='red', marker='o', s=8)  # Smaller marker size

        # Plot title and labels
        ax.set_title(f'Bore: {bore_id}', fontsize=10)
        ax.set_xlabel('Year', fontsize=9)
        ax.set_ylabel('Water Level (mAHD)', fontsize=9)
        ax.legend(fontsize=8)
        ax.grid(True)

    # Remove any extra subplots that are empty
    for j in range(i + 1, rows * 2):  # Iterate over remaining subplots
        fig.delaxes(axes[j // 2][j % 2] if rows > 1 else axes[j % 2])

    # Save the plot to a file, one per page
    plt.savefig(os.path.join(output_dir, f'bores_subplot_A4_page_{page_number}.png'), bbox_inches='tight')
    plt.close()

# Split bores into chunks of 6 (since 3x2 grid fits 6 plots per page)
total_bores = len(unique_bores)
bores_per_page = 6
num_pages = math.ceil(total_bores / bores_per_page)

# Plot each page
for page in range(num_pages):
    start_idx = page * bores_per_page
    end_idx = min(start_idx + bores_per_page, total_bores)
    bores_subset = unique_bores[start_idx:end_idx]
    plot_bore_data_subplot(bores_subset, page + 1)

print(f"All subplot pages saved to '{output_dir}'.")
